# 1. data preprocessing 및 dataset 구축하기

In [62]:
import numpy as np
#import cv2
# show error cases
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from keras.utils import np_utils

num_classes = 2
dataset_dir = "../dataset"
channelName_list = ["BLUE", "GREEN", "IRR", "IRT", "RED"]
headingName_list = ["A", "B", "C", "D"]

def shuffle_data(data, label):
    if len(data) % 4 != 0:
        print("dataNum ERR!!!")
        return data, label, NULL
    
    halfLen = int(len(data)/2)
    quarLen = int(halfLen/2)
    
    idx = np.arange(halfLen)
    #print("S_" + str(idx))
    
    for num in idx:
        if num >= quarLen:
            idx[np.where(idx == num)] = quarLen + num
    
    #print("1_" + str(idx))
    np.random.shuffle(idx)
    #print("2_" + str(idx))

    tmpidx = [quarLen + num for num in idx]
    idx = np.concatenate((idx, tmpidx))
    #print("3_" + str(idx))
    shuffled_data = np.array(data)[np.array(idx)]
    shuffled_label = np.array(label)[np.array(idx)]
    #print("E_" + str(idx))
   
    return shuffled_data, shuffled_label, idx #result_idx
    
def load_data(parent_dir, ext):
    image_A_list = []
    image_B_list = []
    image_C_list = []
    image_D_list = []
    label_A_list = []
    label_B_list = []
    label_C_list = []
    label_D_list = []
    
    image_Dic = {'A' : image_A_list, 'B' : image_B_list, 'C' : image_C_list, 'D' : image_D_list}
    label_Dic = {'A' : label_A_list, 'B' : label_B_list, 'C' : label_C_list, 'D' : label_D_list}
    filePath_list = []
    
    labelNames = os.listdir(parent_dir)
    for labelName in labelNames: # 하위 디렉토리 탐색      (label 폴더 : 0, 1)
        labelDirPath = os.path.join(parent_dir, labelName)
        print(labelName)
        if os.path.isdir(labelDirPath): # 디렉토리인 경우
            headingNames = os.listdir(labelDirPath)
            for headingName in headingNames: # 하위 디렉토리 탐색      (heading 폴더 : A, B, C, D)
                if not(any(headingName in tmpstr for tmpstr in headingName_list)):
                    print("head ERR!!!")
                    continue  # 해당 폴더가 투입면 폴더가 아닌 경우 다시 검색
                #print('_' + headingName)
                headingDirPath = os.path.join(labelDirPath, headingName)

                if os.path.isdir(headingDirPath): # 디렉토리인 경우
                    channelNames = os.listdir(headingDirPath)
                    
                    #로드 리스트 생성
                    loadingImg_list = []
                    loadinglable_list = []
                    loadingName_list = []
                    
                    for channelName in channelNames:  # 하위 디렉토리 탐색      (각 Channel 접근 : R, G, B, IRR, IRT)
                        if not(any(channelName in tmpstr for tmpstr in channelName_list)):
                            print("channel ERR!!!")
                            continue  # 해당 폴더가 채널폴더가 아닌 경우 다시 검색
                        #print('__' + channelName)
                        fileDirPath = os.path.join(headingDirPath, channelName)
                        
                        if os.path.isdir(fileDirPath): # 디렉토리인 경우
                            fileNames = os.listdir(fileDirPath)
                            for fileName in fileNames: #해당폴더 전체 이미지 탐색
                                # 순서대로 새로운 채널에 입력
                                filePath = os.path.join(fileDirPath, fileName)

                                _ext = os.path.splitext(filePath)[-1] # 파일 확장자 확인
                                if _ext == ext:
                                    image = mpimg.imread(filePath)
                                    if image is None:
                                        continue
                                
                                #이미지 크기 정규화
                                image = image.astype(np.float32) / 255.
                                #image = image.reshape(65, 308)
                                
                                #있는지 확인
                                if any(fileName in tmpstr for tmpstr in loadingName_list): #파일이 있는지 없는지 확인
                                    #있으면 dstack
                                    tmpidx = loadingName_list.index(fileName)
                                    shapetmp = loadingImg_list[tmpidx].shape
                                    loadingImg_list[tmpidx] = np.dstack((loadingImg_list[tmpidx], image))
                                    
                                    if channelName == "IRT":
                                        nameListLen = int(len(loadingName_list)/2 -1)
                                        #print(type(nameListLen))
                                        #print(loadingName_list)
                                        for chatchName in loadingName_list[nameListLen:] :
                                            if fileName[5:] in chatchName:
                                                findedName = chatchName
                                        tmp = loadingName_list.index(findedName)
                                        loadingImg_list[tmp] = np.dstack((loadingImg_list[tmp], image))
                                    
                                    #if tmp == 0:
                                    #print("!" + fileName + "\t" + str(tmp) + "\t" + str(len(loadingImg_list)) + "\t" +  str(image.shape) + "\t" +  str(shapetmp) + "\t" +  str(loadingImg_list[tmp].shape))
                                else:
                                    #없으면 append
                                    loadingImg_list.append(image)
                                    #label & name 추가
                                    #loadinglable_list.append(str(len(loadingName_list))+"\t"+labelName + headingName + fileName)
                                    loadinglable_list.append(labelName)
                                    loadingName_list.append(fileName)
                                    #print("?" + fileName)

                    #리스트에 로드된 애들 입력
                    if labelName == '1':
                        image_Dic[headingName] = image_Dic[headingName] + loadingImg_list
                        label_Dic[headingName] = label_Dic[headingName] + loadinglable_list
                    else:
                        image_Dic[headingName] = loadingImg_list
                        label_Dic[headingName] = loadinglable_list
                    for i in range(len(loadingName_list)):
                        filePath_list.append(os.path.join(headingName, loadingName_list[i]))
                    #print(len(image_Dic[headingName]))

    return image_Dic, label_Dic, filePath_list


# data 불러오기
image_Dic, label_Dic, filepath_list = load_data(dataset_dir, '.bmp')

# heading 종류 마다 개수 확인
data_len = len(label_Dic['A'])
for heading in headingName_list:
    if data_len != len(label_Dic[heading]):
        print("DataSize ERR!!!")
        exit()

# dataset 분할
train_size = int(0.7 * data_len)
valid_size = int(0.15 * data_len)
test_size = data_len - train_size - valid_size
print(data_len)
print(train_size, '+', valid_size, '+', test_size, '=', train_size+valid_size+test_size)

# data shuffling
shuffled_image_Dic = {}
shuffled_label_Dic = {}
shuffled_idx_Dic  = {}

for heading in headingName_list:
    shuffled_image_Dic[heading], shuffled_label_Dic[heading], shuffled_idx_Dic[heading] = shuffle_data(image_Dic[heading], label_Dic[heading])

# one-hot vector로 변환
for heading in headingName_list:
    shuffled_label_Dic[heading] = np_utils.to_categorical(shuffled_label_Dic[heading], num_classes)

x_train_Dic = {}
y_train_Dic = {}
x_valid_Dic = {}
y_valid_Dic = {}
x_test_Dic = {}
y_test_Dic = {}
shuffle_indices_Dic = {} # 디버그용
    
# dataset 구축
for heading in headingName_list:
    x_train_Dic[heading] = shuffled_image_Dic[heading][:train_size]
    y_train_Dic[heading] = shuffled_label_Dic[heading][:train_size]
    x_valid_Dic[heading] = shuffled_image_Dic[heading][train_size:data_len-test_size]
    y_valid_Dic[heading] = shuffled_label_Dic[heading][train_size:data_len-test_size]
    x_test_Dic[heading] = shuffled_image_Dic[heading][data_len-test_size:]
    y_test_Dic[heading] = shuffled_label_Dic[heading][data_len-test_size:]
    shuffle_indices_Dic = shuffled_idx_Dic[heading][data_len-test_size:] # 디버그용
    print(heading)
    print(x_train_Dic[heading].shape)
    print(y_train_Dic[heading].shape)
    print(x_test_Dic[heading].shape)
    print(y_test_Dic[heading].shape)

"""
print(len(image_Dic['A']))
print(image_Dic['A'][0].shape)
#print(image_Dic['A'])
#img = image_Dic['A'][0].reshape(325, 308, 1)
img = np.zeros((64, 304))
#print(img.shape)
#for i in range(len(filepath_list)):
#    print(filepath_list[i])


for chan in range(5):
    for i in range(64):
        for j in range(304):
            img[i][j] = image_Dic['A'][5][i][j][chan]
    plt.gray()
    plt.imshow(img)
    plt.show()

print()
for head in headingName_list:
    print(head)
    for imgSize in range(len(shuffled_label_Dic['A'])):
        print(str(int(shuffled_idx_Dic[head][imgSize])%5) + "\t" + str(shuffled_idx_Dic[head][imgSize]) + "\t" + str(shuffled_label_Dic[head][imgSize]))
"""
print()

0
1
20
14 + 3 + 3 = 20
A
(14, 64, 304, 5)
(14, 2)
(3, 64, 304, 5)
(3, 2)
B
(14, 64, 304, 5)
(14, 2)
(3, 64, 304, 5)
(3, 2)
C
(14, 64, 304, 5)
(14, 2)
(3, 64, 304, 5)
(3, 2)
D
(14, 64, 304, 5)
(14, 2)
(3, 64, 304, 5)
(3, 2)



# 2. Model 생성

In [ ]:
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Activation
from keras.models import Sequential, save_model


for heading in headingName_list:
    path_heading_str = "/" + heading + "/"
    projectName_str = "1_full"
    model = Sequential()

    model.add(Conv2D(filters= 16, kernel_size = (3,3), activation='relu', padding='valid', input_shape= x_train.shape[1:]))
    model.add(Conv2D(filters= 16, kernel_size = (3,3), activation='relu', padding='valid'))
    model.add(MaxPooling2D(pool_size= (4, 4)))

    model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    #model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size= (4, 4)))
    
    model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    #model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size= (4, 4)))
    
    model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    #model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size= (4, 4)))

    model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))

    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(10))
    model.add(Activation('softmax'))

    # configure the model
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

    # prints a summary representation of the model
    model.summary()

    # set tensorboard
    from keras.callbacks import TensorBoard
    tbCallback = TensorBoard(log_dir='./tensorboard/' + path_heading_str + projectName_str)
    tbCallback.set_model(model)

    # save the model
    save_model(model, './logs/' + path_heading_str + 'my_model_' + projectName_str + '.hdf5')

    # make parameters checkpoint
    from keras.callbacks import ModelCheckpoint
    checkpoint = ModelCheckpoint(filepath="./logs/" + path_heading_str + "weights.best_" + projectName_str + ".hdf5", verbose=1, 
                                   monitor='loss', save_best_only=True, mode='auto')

    # # set early stopping
    from keras.callbacks import EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)


    # train the model
    model.fit(x_train, y_train,
                    epochs=50,
                    batch_size=256,
                    shuffle=False, # previously shuffled
                    validation_data=(x_valid, y_valid),
                    callbacks = [tbCallback, checkpoint, early_stopping])

    '''
    learning is done at this point
    '''
    score = model.evaluate(x_test, y_test, verbose=0)

    print('Test loss: ', score[0])
    print('Test accuracy: ', score[1])